# OSI Lat Long
### This Notebook uses the XY of the point featureclass and adds Lat Long to each related row in a related table. The FeederID (aka Circuit Number) is also determined for Electric features.
A CSV file is generated that will be used to support a seperate project.

The second half of the Notebook explores the electric consumption data with a [SpatialDataFrame](https://esri.github.io/arcgis-python-api/apidoc/html/arcgis.features.toc.html?highlight=spatialdataframe#spatialdataframe) using the customers table modifed in the first half.

W/WW notes: needs GloabID in customer account tables

In [ ]:
import arcpy
import numpy as np
import pandas as pd

In [ ]:
gdb = r"C:\Users\friendde\Documents\ArcGIS\Projects\OSILatLong\OSILatLong.gdb"

esvcPntDest = r"C:\Users\friendde\Documents\ArcGIS\Projects\OSILatLong\OSILatLong.gdb\eServicePoint"
gsvcPntDest = r"C:\Users\friendde\Documents\ArcGIS\Projects\OSILatLong\OSILatLong.gdb\gMeterSet"
rsvcPntDest = r"C:\Users\friendde\Documents\ArcGIS\Projects\OSILatLong\OSILatLong.gdb\rServicePoint"
wsvcPntDest = r"C:\Users\friendde\Documents\ArcGIS\Projects\OSILatLong\OSILatLong.gdb\ServicePoint"
svcPntDestList = [esvcPntDest,gsvcPntDest,rsvcPntDest,wsvcPntDest]

#svcPntDict = {esvcPntSource:esvcPntDest,gsvcPntSource:gsvcPntDest,rsvcPntSource:rsvcPntDest,wsvcPntSource:wsvcPntDest}

ecustAcctSource = r"C:\GISData\Data\Snapshot\mxElectric.geodatabase\main.eCUSTOMERACCOUNT"
gcustAcctSource = r"C:\GISData\Data\Snapshot\mxGas.geodatabase\main.gCUSTOMERACCOUNT"
rcustAcctSource = r"C:\arcdata\WWW_Extract.gdb\rCustomerAccount"
wcustAcctSource = r"C:\arcdata\WWW_Extract.gdb\wCustomerAccount"
custAcctSourceList = [ecustAcctSource,gcustAcctSource,rcustAcctSource,wcustAcctSource]

ecustAcctDest = r"C:\Users\friendde\Documents\ArcGIS\Projects\OSILatLong\OSILatLong.gdb\eCUSTOMERACCOUNT"
gcustAcctDest = r"C:\Users\friendde\Documents\ArcGIS\Projects\OSILatLong\OSILatLong.gdb\gCUSTOMERACCOUNT"
rcustAcctDest = r"C:\Users\friendde\Documents\ArcGIS\Projects\OSILatLong\OSILatLong.gdb\rCUSTOMERACCOUNT"
wcustAcctDest = r"C:\Users\friendde\Documents\ArcGIS\Projects\OSILatLong\OSILatLong.gdb\wCUSTOMERACCOUNT"
custAcctDestList = [ecustAcctDest,gcustAcctDest,rcustAcctDest,wcustAcctDest]

custAcctDict = {ecustAcctSource:ecustAcctDest,gcustAcctSource:gcustAcctDest,rcustAcctSource:rcustAcctDest,wcustAcctSource:wcustAcctDest}

electFields = ["PHASEDESIGNATION","FEEDERID","TRANSFORMERBANKOBJECTID"]
svcPntDestFlds = ["OID@","GLOBALID","POINT_X","POINT_Y","PHASEDESIGNATION","FEEDERID","TRANSFORMERBANKOBJECTID"]
custAcctDestFlds = ["SERVICEPOINTOBJECTID","GLOBALID","POINT_X","POINT_Y","PHASEDESIGNATION","FeederID","Utility","TRANSFORMERBANKOBJECTID"]
custAcctOutFlds = ["OID@","SvcPntOID","INSTALL_NUM","POINT_X","POINT_Y","PHASEDESIGNATION","FeederID","TRANSFORMERBANKOBJECTID"]
custAnalysis = ["SERVICEPOINTOBJECTID","INSTALL_NUM","POINT_X","POINT_Y","AVGCONSUMPTION","MAXCONSUMPTION"]
custLocations = r"C:\Users\friendde\Documents\ArcGIS\Projects\OSILatLong\OSILatLong.gdb\CustomerLocations"
custAcctFile = r'C:\Users\friendde\Documents\ArcGIS\Projects\OSILatLong\GISAll.csv'

#### Phase Translation between ArcFM and OSI
| Phase | ArcFM | OSI |
|-------|-------|-----|
|   A   |   4   |  1  ||
|   B   |   2   |  2  |
|   C   |   1   |  3  |
|   AB  |   6   |  12 |
|   AC  |   5   |  13 |
|   BC  |   3   |  23 |
|   ABC |   7   |  123|

In [ ]:
# convert ArcFM Phase to OSI Phase
def getPhaseDesignation(phaseDesignation):
        if phaseDesignation is None:
                ph = 0
                return ph
        if phaseDesignation == 1:
                ph = 3
                return ph
        if phaseDesignation == 2:
                ph = 2
                return ph
        if phaseDesignation == 3:
                ph = 23
                return ph
        if phaseDesignation == 4:
                ph = 1
                return ph
        if phaseDesignation == 5:
                ph = 13
                return ph
        if phaseDesignation == 6:
                ph = 12
                return ph
        if phaseDesignation == 7:
                ph = 123
                return ph
def gettxOID(txOID):
        if txOID is None:
                txOID = 0
                return txOID
        else:
            return txOID

def listReplace(lst, _from, _to):
   return([_to if v == _from else v for v in lst])

In [ ]:
arcpy.env.workspace = gdb
arcpy.env.overwriteOutput = True
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference(4326)

In [ ]:
# delete all existing tables in workspace, new tables are created or copied in
fdsList = arcpy.ListDatasets()
for fds in fdsList:
    print(fds)
    arcpy.Delete_management(fds)
tblList = arcpy.ListTables()
for tbl in tblList:
    print(tbl)
    arcpy.Delete_management(tbl)
fcsList = arcpy.ListFeatureClasses()
for fcs in fcsList:
    print(fcs)
    arcpy.Delete_management(fcs)

In [ ]:
# copy in custAcctDict customer tables into workspace
# add fields not normaly in customer tables
for k,v in custAcctDict.items():
    print(f'Copying {k} to {v}')
    arcpy.Copy_management(k,v)
    if arcpy.Exists(v):
        #print(f'Copy Success {v}')
        arcpy.AddField_management(v, "POINT_X", "DOUBLE")
        arcpy.AddField_management(v, "POINT_Y", "DOUBLE")
        arcpy.AddField_management(v, "PHASEDESIGNATION", "LONG")
        arcpy.AddField_management(v, "SERVICEPOINTOBJECTID", "LONG")
        arcpy.AddField_management(v, "Utility", "TEXT")
        arcpy.AddField_management(v, "FeederID", "TEXT")
        arcpy.AddField_management(v, "TRANSFORMERBANKOBJECTID", "LONG")
    else:
        print(f'Copy Failed! {v}')
# add XY coords to service point features
for fc in svcPntDestList:
    if arcpy.Exists(fc):
        print(f'Adding XY to {fc}')
        arcpy.AddXY_management(fc)
        fldList = []
        flds = arcpy.Describe(fc).fields
        for fld in flds:
            fldList.append(fld)
        if electFields not in fldList:
            for electField in electFields:
                print(f'Adding {electField} to {fc}')
                if electField in ["PHASEDESIGNATION","TRANSFORMERBANKOBJECTID"]:
                    arcpy.AddField_management(fc, electField, "LONG")
                else:
                    arcpy.AddField_management(fc, electField, "TEXT")

In [ ]:
for custAcctDest in custAcctDestList:
    fldList = []
    flds = arcpy.Describe(custAcctDest).fields
    if "GlobalID" not in fldList:
        print(f'Adding GlobalID to {custAcctDest}')
        arcpy.AddGlobalIDs_management(custAcctDest)

In [ ]:
# convert ObjectID relationship to GlobalID relationship
rcList = [c.name for c in arcpy.Describe(gdb).children if c.datatype == "RelationshipClass"]
for rc in rcList:
    desc = arcpy.Describe(rc)
    print("%-25s %s" % ("Backward Path Label:", desc.backwardPathLabel))
    print("%-25s %s" % ("Cardinality:", desc.cardinality))
    print("%-25s %s" % ("Class key:", desc.classKey))
    print("%-25s %s" % ("Destination Class Names:", desc.destinationClassNames))
    print("%-25s %s" % ("Forward Path Label:", desc.forwardPathLabel)) 
    print("%-25s %s" % ("Is Attributed:", desc.isAttributed))
    print("%-25s %s" % ("Is Composite:", desc.isComposite)) 
    print("%-25s %s" % ("Is Reflexive:", desc.isReflexive))
    print("%-25s %s" % ("Key Type:", desc.keyType))
    print("%-25s %s" % ("Notification Direction:", desc.notification))
    print("%-25s %s" % ("Origin Class Names:", desc.originClassNames))
    print("\n")

In [ ]:
for rc in rcList:
    print(f'Migrating {rc}')
    arcpy.MigrateRelationshipClass_management(rc)

In [ ]:
edit = arcpy.da.Editor(gdb)
edit.startEditing(False, False)
edit.startOperation()

In [ ]:
for rc in rcList:
    desc = arcpy.Describe(rc)
    if desc.originClassNames[0] == "SAP_INSTALLATION":
        pass
    elif desc.destinationClassNames[0] == "SAP_INSTALLATION":
        pass
    else:
        custAcctFlds = []
        custAcctFlds = listReplace(custAcctDestFlds,"GLOBALID",f"{desc.originClassNames[0]}_GLOBALID")
        with arcpy.da.SearchCursor(desc.originClassNames[0],svcPntDestFlds) as svcpnts:
            print(f"Searching {desc.originClassNames[0]} and updating {desc.destinationClassNames[0]}")
            for svcpnt in svcpnts:
                whereClause = f"{desc.originClassNames[0]}_GLOBALID = '{svcpnt[1]}'"
                with arcpy.da.UpdateCursor(desc.destinationClassNames[0],custAcctFlds,whereClause) as uc:
                    for row in uc:
                        row[0] = svcpnt[0]
                        row[2] = svcpnt[2]
                        row[3] = svcpnt[3]
                        row[4] = getPhaseDesignation(svcpnt[4])
                        row[5] = svcpnt[5]
                        row[6] = desc.destinationClassNames[0][:1]
                        row[7] = gettxOID(svcpnt[6])
                        uc.updateRow(row)

In [ ]:
edit.stopOperation()
edit.stopEditing(True)

In [ ]:
arcpy.CreateTable_management(gdb,"CustomerLocations",ecustAcctDest)

In [ ]:
arcpy.Append_management(custAcctDestList,custLocations,"NO_TEST")

In [ ]:
expression = "setZero(!FeederID!)"
codeblock = """
def setZero(feeder):
    if feeder is None:
        return 0
    else:
        return feeder"""
arcpy.CalculateField_management(custLocations,"FeederId",expression,"PYTHON3",codeblock)

### Use numpy and pandas to export to CSV

Use arcpy [```TableToNumPyArray()```](http://pro.arcgis.com/en/pro-app/arcpy/data-access/tabletonumpyarray.htm)
See also [Working with numpy in ArcGIS](http://pro.arcgis.com/en/pro-app/arcpy/get-started/working-with-numpy-in-arcgis.htm)

nparr = arcpy.da.TableToNumPyArray(custLocations,custAcctOutFlds,skip_nulls=True)
#nparr = arcpy.da.TableToNumPyArray(custAcctDest,custAcctOutFlds,null_value=-9999)
pdarr = pd.DataFrame(nparr)
pdarr.to_csv(custAcctFile,header=False, index=False)

TODO - write file to \\gruadmin.gru.com\fs\Groups\OMS Replacement Project\Documents for OSII\Customer and Premise Files

Ready new numpy array for consumption analysis

remove empty line at end of CSV

In [ ]:
#nparr = arcpy.da.TableToNumPyArray(custLocations,["ServicePointObjectID","POINT_X","POINT_Y","AVGCONSUMPTION","MAXCONSUMPTION","Utility"],skip_nulls=True)

DataFrame.drop_duplicates(subset=None, keep='first', inplace=False

In [ ]:
nparr = arcpy.da.TableToNumPyArray(custLocations,["ObjectID","INSTALL_NUM","POINT_X","POINT_Y","PHASEDESIGNATION","FeederID","Utility","TRANSFORMERBANKOBJECTID"],skip_nulls=True)

In [ ]:
df = pd.DataFrame(nparr)

In [ ]:
df.head()

In [ ]:
df.tail()

Show only rows with a series filter

In [ ]:
df.loc[(df["Utility"]=="w"), ["ObjectID","INSTALL_NUM","POINT_X","POINT_Y","PHASEDESIGNATION","FeederID","Utility"]]

In [ ]:
df.ObjectID

In [ ]:
df = df.astype({'ObjectID':str}, copy=False)

In [ ]:
df.ObjectID

In [ ]:
df.ObjectID = np.where(df.Utility != 'e', df.Utility + df.ObjectID,df.ObjectID)

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.loc[(df.Utility == 'g'), ["ObjectID","INSTALL_NUM","POINT_X","POINT_Y","PHASEDESIGNATION","FeederID","Utility"]]

In [ ]:
#df = df.astype({'INSTALL_NUM':str}, copy=True)
df2 = df.astype(str, copy=True)
df2.INSTALL_NUM

#df.groupby(df.columns.tolist(),as_index=False).size()
grouped = df.groupby('Utility')
for name,group in grouped:
    print(name)
    #print(group)
    #df.groupby('INSTALL_NUM').filter(lambda x: len(x) > 1).drop_duplicates(subset='INSTALL_NUM')
    #df2.groupby(['INSTALL_NUM', 'Utility']).filter(lambda x: x if len(x) > 1 and 'Utility' == name else None)
    #print(df.groupby('INSTALL_NUM').filter(lambda x: x if len(x) > 1 and 'Utility' == name else None))
    #print(df.groupby('INSTALL_NUM').filter(lambda x: len(x) > 1 and 'Utility' == name))
    print(df.groupby('INSTALL_NUM').filter(lambda x: len(x) > 1)).drop_duplicates(subset='INSTALL_NUM')

In [ ]:
df.drop_duplicates(subset='INSTALL_NUM', keep='first', inplace=True)

In [ ]:
df.groupby('INSTALL_NUM').filter(lambda x: len(x) > 1)

In [ ]:
df.groupby(['INSTALL_NUM']).size().reset_index(name='count')

In [ ]:
df.drop(['Utility'], axis=1, inplace=True)

In [ ]:
df.tail()

In [ ]:
df.to_csv(custAcctFile,header=False, index=True)

[```gis.features.SpatialDataFrame()```](https://esri.github.io/arcgis-python-api/apidoc/html/arcgis.features.toc.html?highlight=spatialdataframe#arcgis.features.SpatialDataFrame.from_xy)

In [ ]:
from arcgis.features import SpatialDataFrame
from arcgis.gis import GIS
from getpass import getpass
from IPython.display import display

In [ ]:
sdf = SpatialDataFrame.from_xy(df,"POINT_X","POINT_Y")
gis = GIS(arcpy.GetActivePortalURL(), username=input("Enter User Name "), password=(getpass()))
#gis = GIS()
#portalDesc = arcpy.GetPortalDescription()
# search and list all items owned by connected user
#query=f'owner:{portalDesc["user"]["username"]} AND title:CW BaseMap'
#itemType="Feature Layer"
#sortField="title"
#sortOrder="asc"
# default max__items is 10
#maxItems=100
#m = gis.content.search(query,itemType,sortField,sortOrder,maxItems)

In [ ]:
consumptionLyr = gis.content.import_data(sdf)

In [ ]:
m = gis.map('Gainesville,FL')

In [ ]:
m.add_layer(sdf,options={"renderer":"ClassedSizeRenderer","field_name":"MAXCONSUMPTION"})

In [ ]:
m.add_layer(consumptionLyr,options={"renderer":"ClassedSizeRenderer","field_name":"MAXCONSUMPTION"})

In [ ]:
m